In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI管道：使用预构建的Google Cloud管道组件进行自定义培训

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/custom_model_training_and_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/custom_model_training_and_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/custom_model_training_and_batch_prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用预构建的 Google Cloud Pipeline 组件和 Vertex AI Pipelines 进行自定义训练。

了解有关 [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) 和 [自定义训练组件](https://cloud.google.com/vertex-ai/docs/training/create-training-pipeline) 的更多信息。

### 目标

在本教程中，您将学习如何使用Vertex AI Pipelines和Google Cloud Pipeline组件来构建自定义模型。

本教程使用以下Google Cloud ML服务：

- Vertex AI Pipelines
- Google Cloud Pipeline组件
- Vertex AI Training
- Vertex AI模型资源
- Vertex AI端点资源

执行的步骤包括：

- 创建一个KFP管道：
    - 训练一个自定义模型。
    - 将训练好的模型上传为模型资源。
    - 创建一个端点资源。
    - 部署模型资源到端点资源。
    - 进行批量预测请求。

了解更多关于 [Google Cloud Pipeline components](https://cloud.google.com/vertex-ai/docs/pipelines/build-pipeline)。

### 数据集

本教程使用的数据集是[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已嵌入到TensorFlow中。训练模型预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船或卡车。

费用

本教程使用 Google Cloud 的应收组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成费用估算。

## 安装

安装执行此笔记本所需的包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components


! pip3 install --upgrade --force-reinstall tensorflow kfp google-cloud-aiplatform google-cloud-storage google-cloud-pipeline-components -q

只有合作才行：取消以下单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的Google云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)

4. 如果您在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云帐号

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面的相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经通过验证。

**2. 本地JupyterLab实例，请取消注释并运行:**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

服务账户或其他

请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples给您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶，用于存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶还不存在：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

#### 服务账户

**如果您不知道您的服务账户**，请尝试使用`gcloud`命令在执行下面的第二个单元格中获取您的服务账户。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### 为 Vertex AI Pipelines 设置服务账号访问权限

运行以下命令，将您的服务账号授予读取和写入管道工件的权限，这些工件位于您在上一步中创建的存储桶中 -- 您只需为每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.v1.custom_job import utils
from kfp import compiler, dsl
from kfp.dsl import component

#### Vertex AI Pipelines 常量

为 Vertex AI Pipelines 设置以下常量：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/bikes_weather".format(BUCKET_URI)

初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，并将4个Nvidia Telsa K80 GPU分配给每个VM，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以在CPU上运行容器映像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的信息。

*注意*：在TF 2.3之前的GPU支持下，将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由生成在serving function中的静态图操作引起的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)

DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2-5"

if TRAIN_GPU:
    TRAIN_VERSION = "tf-gpu.{}".format(TF)
else:
    TRAIN_VERSION = "tf-cpu.{}".format(TF)
if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个 vCPU 3.75GB 的内存。
     - `n1-highmem`: 每个 vCPU 6.5GB 的内存。
     - `n1-highcpu`: 每个 vCPU 0.9GB 的内存。
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96] 的数量。

*注意：以下内容不支持用于训练*：

 - `标准`：2个 vCPUs
 - `高CPU`：2、4和8个 vCPUs

*注意：您还可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

教程

现在你已经准备好开始为CIFAR10创建自己的自定义模型和训练。

### 检查培训包

#### 包布局

在开始培训之前，您将查看一个Python包是如何为自定义培训任务组装的。解压后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是在Docker镜像的操作环境中安装包的说明。

文件`trainer/task.py`是执行自定义培训任务的Python脚本。*注意*，当我们在worker pool规范中引用它时，我们将目录斜杠替换为一个点（`trainer.task`）并且删除文件后缀（`.py`）。

#### 包装配

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为训练自定义模型创建自定义组件

接下来，创建一个轻量级的Python函数组件，用于训练CIFAR10图像分类模型。

In [ ]:
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10


@component(
    base_image="tensorflow/tensorflow:latest",
    packages_to_install=["tensorflow_datasets", "opencv-python-headless"],
)
def custom_train_model(
    model_dir: str,
    lr: float = 0.01,
    epochs: int = 10,
    steps: int = 200,
    distribute: str = "single",
):

    import faulthandler
    import os
    import sys

    import tensorflow as tf
    import tensorflow_datasets as tfds
    from tensorflow.python.client import device_lib

    faulthandler.enable()
    tfds.disable_progress_bar()

    print("Component start")

    print("Python Version = {}".format(sys.version))
    print("TensorFlow Version = {}".format(tf.__version__))
    print("TF_CONFIG = {}".format(os.environ.get("TF_CONFIG", "Not found")))
    print("DEVICES", device_lib.list_local_devices())

    # Single Machine, single compute device
    if distribute == "single":
        if tf.test.is_gpu_available():
            strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
        else:
            strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
    # Single Machine, multiple compute device
    elif distribute == "mirror":
        strategy = tf.distribute.MirroredStrategy()
    # Multiple Machine, multiple compute device
    elif distribute == "multi":
        strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

    # Multi-worker configuration
    print("num_replicas_in_sync = {}".format(strategy.num_replicas_in_sync))

    # Preparing dataset
    BUFFER_SIZE = 10000
    BATCH_SIZE = 64

    def make_datasets_unbatched():

        # Scaling CIFAR10 data from (0, 255] to (0., 1.]
        def scale(image, label):
            image = tf.cast(image, tf.float32)
            image /= 255.0
            return image, label

        datasets, info = tfds.load(name="cifar10", with_info=True, as_supervised=True)
        return datasets["train"].map(scale).cache().shuffle(BUFFER_SIZE).repeat()

    # Build the Keras model
    def build_and_compile_cnn_model(lr: int = 0.01):
        model = tf.keras.Sequential(
            [
                tf.keras.layers.Conv2D(
                    32, 3, activation="relu", input_shape=(32, 32, 3)
                ),
                tf.keras.layers.MaxPooling2D(),
                tf.keras.layers.Conv2D(32, 3, activation="relu"),
                tf.keras.layers.MaxPooling2D(),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(10, activation="softmax"),
            ]
        )
        model.compile(
            loss=tf.keras.losses.sparse_categorical_crossentropy,
            optimizer=tf.keras.optimizers.SGD(learning_rate=lr),
            metrics=["accuracy"],
        )
        return model

    # Train the model
    NUM_WORKERS = strategy.num_replicas_in_sync
    # Here the batch size scales up by number of workers since
    # `tf.data.Dataset.batch` expects the global batch size.
    GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
    train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

    with strategy.scope():
        # Creation of dataset, and model building/compiling need to be within
        # `strategy.scope()`.
        model = build_and_compile_cnn_model(lr)

    model.fit(x=train_dataset, epochs=epochs, steps_per_epoch=steps)
    model.save(model_dir)

    model_path_to_deploy = model_dir

    # Load the saved model
    local_model = tf.keras.models.load_model(model_dir)

    # Load evaluation data
    import numpy as np
    from tensorflow.keras.datasets import cifar10

    (_, _), (x_test, y_test) = cifar10.load_data()
    x_test = (x_test / 255.0).astype(np.float32)

    print(x_test.shape, y_test.shape)

    # Perform the model evaluation
    local_model.evaluate(x_test, y_test)

    # Serving function for image data
    CONCRETE_INPUT = "numpy_inputs"

    def _preprocess(bytes_input):
        decoded = tf.io.decode_jpeg(bytes_input, channels=3)
        decoded = tf.image.convert_image_dtype(decoded, tf.float32)
        resized = tf.image.resize(decoded, size=(32, 32))
        return resized

    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def preprocess_fn(bytes_inputs):
        decoded_images = tf.map_fn(
            _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
        )
        return {
            CONCRETE_INPUT: decoded_images
        }  # User needs to make sure the key matches model's input

    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def serving_fn(bytes_inputs):
        images = preprocess_fn(bytes_inputs)
        prob = m_call(**images)
        return prob

    m_call = tf.function(local_model.call).get_concrete_function(
        [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
    )

    tf.saved_model.save(
        local_model, model_path_to_deploy, signatures={"serving_default": serving_fn}
    )

    # Get the serving function signature
    loaded = tf.saved_model.load(model_path_to_deploy)

    serving_input = list(
        loaded.signatures["serving_default"].structured_input_signature[1].keys()
    )[0]
    print("Serving function input:", serving_input)

    # Get test items
    test_image_1 = x_test[0]
    test_image_2 = x_test[1]
    print(test_image_1.shape)

    BUCKET_URI = model_dir + "/test"

    import cv2

    cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
    cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

    print("Writing jpg files")

    # Copy test item(s)
    # For the batch prediction, copy the test items over to your Cloud Storage bucket.
    test_item_1 = BUCKET_URI + "/tmp1.jpg"
    test_item_2 = BUCKET_URI + "/tmp2.jpg"

    with tf.io.gfile.GFile(test_item_1, "wb") as w:
        with tf.io.gfile.GFile("tmp1.jpg", "rb") as r:
            bytes = r.read()
            w.write(bytes)

    with tf.io.gfile.GFile(test_item_2, "wb") as w:
        with tf.io.gfile.GFile("tmp2.jpg", "rb") as r:
            bytes = r.read()
            w.write(bytes)

    # Make the batch input file
    import base64
    import json

    gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
    with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
        bytes = tf.io.read_file(test_item_1)
        b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
        data = {serving_input: {"b64": b64str}}
        f.write(json.dumps(data) + "\n")
        bytes = tf.io.read_file(test_item_2)
        b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
        data = {serving_input: {"b64": b64str}}
        f.write(json.dumps(data) + "\n")

### 将组件转换为一个 Vertex AI 自定义作业

接下来，使用`create_custom_training_job_op_from_component`方法，将自定义组件转换为一个 Vertex AI 预构建自定义作业组件。

**replica_count :** 批处理操作可扩展到的机器副本数。仅在设置了machine_type时使用。默认值为10。

In [ ]:
custom_job_distributed_training_op = utils.create_custom_training_job_op_from_component(
    custom_train_model, replica_count=1
)

定义自定义训练工作流程

接下来，定义包含以下任务的管道作业：

- 训练自定义模型。
- 将模型上传到Verex AI模型资源。
- 执行批处理预测。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"


@dsl.pipeline(name="custom-model-training-sample-pipeline")
def pipeline(
    model_dir: str = MODEL_DIR,
    lr: float = 0.01,
    epochs: int = 10,
    steps: int = 200,
    distribute: str = "single",
):
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.batch_predict_job import \
        ModelBatchPredictOp
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.dsl import importer_node

    custom_producer_task = custom_job_distributed_training_op(
        model_dir=model_dir,
        lr=lr,
        epochs=epochs,
        steps=steps,
        distribute=distribute,
        project=PROJECT_ID,
        location=REGION,
        base_output_directory=PIPELINE_ROOT,
    )

    unmanaged_model_importer = importer_node.importer(
        artifact_uri=model_dir,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-3:latest"
            }
        },
    )

    model_upload_op = ModelUploadOp(
        project=PROJECT_ID,
        display_name="model_display_name",
        unmanaged_container_model=unmanaged_model_importer.outputs["artifact"],
    )
    model_upload_op.after(custom_producer_task)

    batch_predict_op = ModelBatchPredictOp(
        project=PROJECT_ID,
        job_display_name="batch_predict_job",
        model=model_upload_op.outputs["model"],
        gcs_source_uris=[MODEL_DIR + "/test/test.jsonl"],
        gcs_destination_output_uri_prefix=PIPELINE_ROOT,
        instances_format="jsonl",
        predictions_format="jsonl",
        model_parameters={},
        machine_type=DEPLOY_COMPUTE,
        starting_replica_count=1,
        max_replica_count=1,
    )

    batch_predict_op.after(model_upload_op)

### 编译和运行管道
接下来，编译管道成为一个DAG，然后执行它。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="custom_model_training_spec.yaml"
)

DISPLAY_NAME = "cifar10"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="custom_model_training_spec.yaml",
    pipeline_root=PIPELINE_ROOT,
)

job.run(service_account=SERVICE_ACCOUNT)

! rm custom_model_training_spec.json

查看自定义训练管道结果

最后，您可以查看管道中每个任务的工件输出。

In [ ]:
import json

import tensorflow as tf

PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVAL_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        if tf.io.gfile.exists(EXECUTE_OUTPUT):
            ! gsutil cat $EXECUTE_OUTPUT
            return EXECUTE_OUTPUT
        elif tf.io.gfile.exists(GCP_RESOURCES):
            ! gsutil cat $GCP_RESOURCES
            return GCP_RESOURCES
        elif tf.io.gfile.exists(EVAL_METRICS):
            ! gsutil cat $EVAL_METRICS
            return EVAL_METRICS

    return None


print("model-upload")
artifacts = print_pipeline_output(job, "model-upload")
print("\n")
output = !gsutil cat $artifacts
print(output)
output = json.loads(output[0])
model_id = output["artifacts"]["model"]["artifacts"][0]["metadata"]["resourceName"]
print("model-batch-predict")
artifacts = print_pipeline_output(job, "model-batch-predict")
print("\n")
output = !gsutil cat $artifacts
output = json.loads(output[0])
batch_job_id = output["artifacts"]["batchpredictionjob"]["artifacts"][0]["metadata"][
    "resourceName"
]

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

job.delete()

model = aip.Model(model_id)
model.delete()

batch_job = aip.BatchPredictionJob(batch_job_id)
batch_job.delete()


delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}